# ICTDS4 Gezamenlijke notebook

In deze notebook staat alle code dat gemaakt is. 

De code dat hierinstaat is de samenvatting per type, om een beter beeld te krijgen naar de waarom's en hoe we het gedaan hebben refereren we naar de aparte jupyter notebooks in de models folder.

Alle imports nodig om de code te laten draaien

In [ ]:
# Imports
import datetime
import json
import matplotlib.pyplot as plt
import nltk
import numpy as np
import openai
import pandas as pd
import platform
import re
import sys
import tensorflow as tf
import tqdm

from google.api_core import retry
from keras import layers
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from tqdm.auto import tqdm

# Group: Data Processing
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Group: Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Group: Machine Learning
import sklearn as sk
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

# Group: Deep Learning
import keras
import tensorflow as tf
from keras import layers

# Group: Database
import sqlite3
import sqlalchemy

# Group: Miscellaneous
import datetime
import joblib
import openai

# Group: Google APIs
import google.generativeai as palm
from google.api_core import retry

# Group: Utility
import sys
import platform
from tqdm.auto import tqdm


## Data preparation

In [ ]:
# get the date of yesterday
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday_str = yesterday.strftime('%Y-%m-%d')

# create the URL with the date of yesterday
url = f'https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/?date_received_max={yesterday_str}&date_received_min=2011-12-01&field=all&format=csv&lens=product&no_aggs=true&product=Mortgage&size=375533&sub_lens=sub_product&trend_depth=5&trend_interval=month'

parse_dates = ['Date received', 'Date sent to company']
dtypes = {'Date received': str,
          'Product': "category",
          'Sub-product': "category",
          'Issue': "category",
          'Sub-issue': "category",
          'Consumer complaint narrative': str,
          'Company public response': str,
          'Company': "category",
          'State': "category",
          'ZIP code': str,
          'Tags': "category",
          'Consumer consent provided?': str,
          'Submitted via': "category",
          'Date sent to company': str,
          'Company response to consumer': str,
          'Timely response?': str,
          'Consumer disputed?': str,
          'Complaint ID': int}

DS1_data = pd.read_csv(url, low_memory=False, dtype=dtypes, parse_dates=parse_dates)
DS1_data[['Timely response?', 'Consumer disputed?']] = DS1_data[['Consumer disputed?', 'Timely response?']].replace(
    {'Yes': True, 'No': False}).astype(bool)
DS1_data['Consumer consent provided?'] = DS1_data['Consumer consent provided?'].replace(
    {'Consent provided': True, '': False}).astype(bool)
DS1_data = DS1_data[pd.notnull(DS1_data['Consumer complaint narrative'])]  # Drops the missing rows with no  complaint.
DS1_data.drop(columns=['Sub-issue', "Product"])

db = sqlalchemy.create_engine('sqlite:///StaterData.db')
DS1_data.to_sql("mortgage complaints", db, if_exists="replace")

## Data cleaning

In [ ]:
# Constants
conn = sqlite3.connect('StaterData.db')
query = "SELECT * FROM 'mortgage complaints'"
dtypes = {
    'Date received': str,
    'Product': "category",
    'Sub-product': "category",
    'Issue': "category",
    'Sub-issue':"category",
    'Consumer complaint narrative':str,
    'Company public response':str,
    'Company':"category",
    'State':"category",
    'ZIP code':str,
    'Tags':"category",
    'Consumer consent provided?':str,
    'Submitted via':"category",
    'Date sent to company':str,
    'Company response to consumer':str,
    'Timely response?':str,
    'Consumer disputed?':str,
    'Complaint ID':int
}

# Remove stopwords and return the text
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return " ".join([token for token in nltk.word_tokenize(text) if token.lower() not in stop_words])

# Clean the text
def clean_text(text):
    # Remove numerical values
    text = re.sub(r'\d+', '', text)

    # Remove punctuation marks
    text = re.sub(r'[^\w\s]', '', text)

    # Remove links and URLs
    text = re.sub(r'http\S+', '', text)

    # Remove leading/trailing white space and convert to lowercase
    text = text.strip().lower()

    return text

# Lemmitize the word based on its part of speech (POS) tag
def lemmatize_word(word, tag, lemmatizer):
    wn_tag = None

    # Map POS tag to WordNet POS tag
    if tag.startswith('J'):
        # Adjective
        wn_tag = 'a'
    elif tag.startswith('V'):
        # Verb
        wn_tag = 'v'
    elif tag.startswith('N'):
        # Noun
        wn_tag = 'n'
    elif tag.startswith('R'):
        # Adverb
        wn_tag = 'r'

    # Lemmatize the word
    if wn_tag:
        lemma = lemmatizer.lemmatize(word, wn_tag)
    else:
        lemma = word

    return lemma

# Lemmatize the sentence that is already tokenized
def lemmatize_sentence(text):
    lemmatizer = WordNetLemmatizer()

    # Tokenize the sentence into words
    tokens = nltk.word_tokenize(text)

    # Part-of-speech (POS) tag each word
    pos_tags = nltk.pos_tag(tokens)

    # Lemmatize each word based on its POS tag
    lemmas = [lemmatize_word(word, tag, lemmatizer) for word, tag in pos_tags]

    # Join the lemmas back into a sentence
    lemmatized_sentence = ' '.join(lemmas)

    return lemmatized_sentence

def remove_non_alphabetica_char_and_x(text):
    tokenized_text = nltk.word_tokenize(text)

    # Remove non alphabetical characters
    alphabetical_text = [re.sub('[^a-zA-Z]+', '', word) for word in tokenized_text]

    # Remove x from the text with regex
    alphabetical_text = [word for word in alphabetical_text if not re.match('^x+$', word)]
    return ' '.join(alphabetical_text)

def clean_complaint(complaint):
    cleaned_complaint = remove_stopwords(complaint)
    cleaned_complaint = clean_text(cleaned_complaint)
    cleaned_complaint = lemmatize_sentence(cleaned_complaint)
    cleaned_complaint = remove_non_alphabetica_char_and_x(cleaned_complaint)
    return cleaned_complaint

# Read data
data = pd.read_sql_query(query, conn, dtype=dtypes)
df = data.copy()

# Clean consumer complaint
df['Clean consumer complaint'] = df['Consumer complaint narrative'].apply(lambda x: clean_complaint(x))

df = df[['Complaint ID', 'Issue', 'Consumer complaint narrative', 'Clean consumer complaint']]

# Save data in csv
df.to_csv('StaterData.csv', index=False)
print("Data is saved in csv")

# Traditionele methodes

## Decision tree

In [ ]:
# Imports
import pandas as pd

from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score


# Read csv, StaterData

df = pd.read_csv('StaterData.csv')

# Define your features and target, with x as text_column which contains the complaint in the dataset. And target_column which contains the target variable
X = df['Clean consumer complaint']  
y = df['Issue']                        

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectorize your text data using a bag-of-words approach
vectorizer = TfidfVectorizer(stop_words='english',token_pattern=r'\b[a-zA-Z]+\b')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

feature_names = vectorizer.get_feature_names_out()
class_names = df['Issue']

# Gridsearch Define the parameter grid
best_params = {
    'criterion': 'gini',
    'max_depth': 14,
    'min_samples_split': 2,
    'min_samples_leaf': 1
    }

# Create a Decision tree model
model = DecisionTreeClassifier(random_state=2, **best_params)

# Train the model on the training data
model.fit(X_train, y_train)

# Predict the target variable for the testing data
y_pred = model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

# function to classify a complaint
def questionDecisionTree(text):
    question_decisionTree = vectorizer.transform([text])
    prediction = model.predict(question_decisionTree)
    return prediction[0]

## Random Forest

In [ ]:
# Read cleaned dataset created from complaintCleaner.py
data = pd.read_csv('StaterData.csv')

# Define your features and target
X = data['Clean consumer complaint']  # text_column is the name of the column in your dataset that contains the text data
y = data['Issue']  # target_column is the name of the column in your dataset that contains the target variable

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Define params from GridSearch which performed the best
param_grid = {
    'n_estimators': 200
}

# Vectorize your text data using TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(random_state=2)

# Set the params for the model
rf_model.set_params(**param_grid)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict the target variable for the testing data
y_pred = rf_model.predict(X_test)

def text_to_prediction_random_forest(text):
    custom_text_bow = vectorizer.transform([text])
    predicted_issue = rf_model.predict(custom_text_bow)

    return predicted_issue

# Define your features and target
X = data['Consumer complaint narrative']  # text_column is the name of the column in your dataset that contains the text data
y = data['Issue']  # target_column is the name of the column in your dataset that contains the target variable

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectorize your text data using a bag-of-words approach
vectorizer = TfidfVectorizer(stop_words=['english'])
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=2)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_

# Create a DataFrame with feature names and importances
feature_importances = pd.DataFrame({'feature': vectorizer.get_feature_names(), 'importance': importances})

# Sort the DataFrame by importance
feature_importances = feature_importances.sort_values('importance', ascending=False)

top_features = feature_importances.head(10)['feature'].values.tolist()

# Define your features and target
X = data['Consumer complaint narrative']  # text_column is the name of the column in your dataset that contains the text data
y = data['Issue']  # target_column is the name of the column in your dataset that contains the target variable

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectorize your text data using a bag-of-words approach
vectorizer = TfidfVectorizer(stop_words='english', vocabulary=top_features)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=2)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict the target variable for the testing data
y_pred = rf_model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
f1score = f1_score(y_test, y_pred, average='macro')

# # Define the parameter grid to search over
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
# }
#
# # Create a Random Forest model
# rf_model = RandomForestClassifier(random_state=2)
#
# # Create a GridSearchCV object
# grid_search = GridSearchCV(
#     estimator=rf_model,
#     param_grid=param_grid,
#     cv=5,  # number of folds for cross-validation
#     scoring='accuracy',
#     # n_jobs=-1,  # use all available CPU cores
# )
#
# # Fit the grid search to the training data
# grid_search.fit(X_train, y_train)
# 
# # Print the best hyperparameters and the corresponding F1 score
# print('Best hyperparameters:', grid_search.best_params_)
# print('Best accuracy score:', grid_search.best_score_)

## Multionominal logistic regression

In [ ]:
csv_file = 'StaterData.csv'
dtypes = {
    'Date received': str,
    'Product': "category",
    'Sub-product': "category",
    'Issue': "category",
    'Sub-issue':"category",
    'Consumer complaint narrative':str,
    'Company public response':str,
    'Company':"category",
    'State':"category",
    'ZIP code':str,
    'Tags':"category",
    'Consumer consent provided?':str,
    'Submitted via':"category",
    'Date sent to company':str,
    'Company response to consumer':str,
    'Timely response?':str,
    'Consumer disputed?':str,
    'Complaint ID':int,
    'Clean consumer complaint':str,
}

# Read data from csv
data = pd.read_csv(csv_file, dtype=dtypes)
df = data[['Clean consumer complaint', 'Issue']].copy()

# Split the data into training and testing sets: x = complaint and y = issue
X_train, X_test, y_train, y_test = train_test_split(df['Clean consumer complaint'], df['Issue'], test_size=0.3, random_state=2)

# Vectorize the data with tfidf
vectorizer = TfidfVectorizer(stop_words='english', token_pattern=r'\b[a-zA-Z]+\b')
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Create multinomial logistic regression
logregression = LogisticRegression(C=1, multi_class='multinomial', n_jobs=4, random_state=2, solver='saga')

# Fit and predict
logregression.fit(X_train_tfidf, y_train)
y_pred = logregression.predict(X_test_tfidf)

# function to classify a complaint
def ask_question(text):
    question_tfidf = vectorizer.transform([text])
    prediction = logregression.predict(question_tfidf)
    return prediction[0]

## State Vector Machine

In [ ]:
# load data
data = pd.read_csv("StaterData.csv")

# split data
train_data, test_data, train_labels, test_labels = train_test_split(data['Clean consumer complaint'], data['Issue'], test_size=0.1, random_state=2)

# vectorize data
vectorizer = TfidfVectorizer(
    stop_words="english",
    token_pattern=r'\b[a-zA-Z]+\b',
    analyzer="word",
    use_idf=True
)

# Vectorize data
vectorizer.fit(train_data)

# If needed, use grid search to find best parameters
# clf = SVC(kernel='linear')

# parameters = {
#     'C': [0.1, 1, 10, 100],
#     'gamma': ['scale', 'auto'],
#     'class_weight': ['balanced', None],
# }

# grid_search = GridSearchCV(clf, parameters, cv=5)
# grid_search.fit(train_tfidf_vectors, train_labels)

# print("Best parameters:", grid_search.best_params_)
# print("Best score:", grid_search.best_score_)

# Set model parameters
clf = SVC(C=10, class_weight='balanced', gamma='scale', kernel='linear')

# Fit model
train_tfidf_vectors = vectorizer.fit_transform(train_data)

# Transform test data
test_tfidf_vectors = vectorizer.transform(test_data)

# Fit model
clf.fit(train_tfidf_vectors, train_labels)

# Predict
pred_labels = clf.predict(test_tfidf_vectors)

# Evaluate
accuracy = accuracy_score(test_labels, pred_labels, normalize=True)
count = data.shape[0] // 1000

# Print the results
print(f"Accuracy: {accuracy * 100:.2f}% with({count}k samples)")

## TF-IDF

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('words')

# Define the data types for each column
dtypes = {
    'Date received': str,
    'Product': "category",
    'Sub-product': "category",
    'Issue': "category",
    'Sub-issue': "category",
    'Consumer complaint narrative': str,
    'Company public response': str,
    'Company': "category",
    'State': "category",
    'ZIP code': str,
    'Tags': "category",
    'Consumer consent provided?': str,
    'Submitted via': "category",
    'Date sent to company': str,
    'Company response to consumer': str,
    'Timely response?': str,
    'Consumer disputed?': str,
    'Complaint ID': int
}

# Define the columns to parse as dates
parse_dates = ['Product', 'Date received', 'Date sent to company']

# Read the CSV file with specified data types and parse dates
DS1_data = pd.read_csv("Data/complaints-2023-04-25_05_07.csv", low_memory=False, dtype=dtypes, parse_dates=parse_dates)

# Convert 'Timely response?' and 'Consumer disputed?' columns to boolean values
DS1_data[['Timely response?', 'Consumer disputed?']] = DS1_data[['Timely response?', 'Consumer disputed?']].replace({'Yes': True, 'No': False}).astype(bool)

# Convert 'Consumer consent provided?' column to boolean values
DS1_data['Consumer consent provided?'] = DS1_data['Consumer consent provided?'].replace({'Consent provided': True, '': False}).astype(bool)

# Drop rows with missing complaint narratives
DS1_data.dropna(subset=['Consumer complaint narrative'], inplace=True)

# Drop the 'Sub-issue' column as it is not needed
DS1_data.drop(columns=['Sub-issue'], inplace=True)

# Replace alle X occurences with emty strings, to avoid it from being the most important word.
DS1_data['Consumer complaint narrative'] = DS1_data['Consumer complaint narrative'].str.replace('X', '')

# Calculate the normalized count of issue categories
IssueCountNormalized = DS1_data['Issue'].value_counts(normalize=True)

# Create a TfidfVectorizer with optimized settings
vectorizer = TfidfVectorizer(stop_words='english',              # Exclude common English words
                             token_pattern=r'\b[a-zA-Z]+\b',    # Consider only alphabetic tokens
                             analyzer='word',                   # Analyze at the word level
                             use_idf=True,                      # Apply inverse document frequency weighting
                             smooth_idf=True,                   # Apply smoothing to idf weights
                             strip_accents='ascii',
                             min_df=2,
                             norm='l2')

# Fit and transform the vectorizer to get an score per word in an array returned
Vectorized_Data = vectorizer.fit_transform(DS1_data['Consumer complaint narrative'])

# Add the scores array back into the dataframe.
DS1_data['TF-IDF scores'] = list(Vectorized_Data.toarray())

# Defines the dictionary of english words
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

# Gets the list of words from the vectorizer
feature_names = vectorizer.get_feature_names_out()

# Get unique values from the "Issue" column
unique_issues = DS1_data['Issue'].unique()

# Concatenate the unique issues into a single string
all_issues = ' '.join(unique_issues)

# Split the concatenated string into individual words
all_words = all_issues.split()

# Get unique words
Mortgage_Terms = set(all_words)

# Set an empty top 3 words list.
top_words = []


for i in range(Vectorized_Data.shape[0]):
    print(i)
    # Get the array with scores from this row
    row_scores = Vectorized_Data[i].toarray()[0]

    # Generate a dictionary with each word and then the Scores
    scores_dict = {name: score for name, score in zip(feature_names,row_scores)}

    # Loop over all the words and adjust the score
    for name in feature_names:
        # If term is in categoryname, add to score
        if name in Mortgage_Terms:
            scores_dict[name] *= (1 + 0.02*len(name) + 0.2)
        else:
            scores_dict[name] *= (1 + 0.02*len(name))

    # Create a new array of adjusted scores in the same order as the feature names
    adjusted_scores = np.zeros(len(feature_names))
    for i, term in enumerate(feature_names):
        adjusted_scores[i] = scores_dict[term]

    #Set the top 3 to a empty list.
    Top3Words = []

    # Get the index of the highest scoring word
    max_index = adjusted_scores.argmax()

    # Iterate until 3 top words found or list of words empty
    while max_index != 0 and len(Top3Words) <= 2:
        #Check if there are any vowels in the topword, if not select new word
        while True:
            vowels = {"a", "e", "i", "o", "u", "A", "E", "I", "O", "U"}
            # Get the corresponding word
            top_word = feature_names[max_index]
            if any(char in vowels for char in feature_names[max_index]) and top_word in english_vocab:
                Top3Words.append(top_word)
            break

        #Sets the current score of this word to 0 to select the second most popular word
        adjusted_scores[max_index] = 0
        max_index = adjusted_scores.argmax()
    # Voeg het bijbehorende woord toe aan de lijst van top_words
    top_words.append(Top3Words)

# Add the list of topwords to the dataframe
DS1_data['top_word'] = top_words

# Create a TfidfVectorizer with optimized settings
vectorizer = TfidfVectorizer(stop_words='english',              # Exclude common English words
                             token_pattern=r'\b[a-zA-Z]+\b',    # Consider only alphabetic tokens
                             analyzer='word',                   # Analyze at the word level
                             use_idf=True,                      # Apply inverse document frequency weighting
                             smooth_idf=True,                   # Apply smoothing to idf weights
                             strip_accents='ascii',
                             min_df=2,
                             norm='l2')

# Fit and transform the vectorizer to get an score per word in an array returned
Vectorized_Data = vectorizer.fit_transform(data['Consumer complaint narrative'])

feature_names = vectorizer.get_feature_names_out()

# Defines the dictionary of english words
english_vocab = set(w.lower() for w in nltk.corpus.words.words())

# Calculate the normalized count of issue categories
IssueCountNormalized = data['Issue'].value_counts(normalize=True)

# Get unique values from the "Issue" column
unique_issues = data['Issue'].unique()

# Concatenate the unique issues into a single string
all_issues = ' '.join(unique_issues)

# Split the concatenated string into individual words
all_words = all_issues.split()

# Get unique words
Mortgage_Terms = set(all_words)

# Function creates an array of every word in text and gives it an score
def tfidf_custom_scoring(input_text):
    # Fit the data to the vectorizer
    vectorizer.fit(data['Consumer complaint narrative'])

    # Adds the input from the user to the fit using an transform
    transformed_data = vectorizer.transform([input_text])

    # Get all the feature name(words instead of numbers) corresponding to the array
    feature_names = vectorizer.get_feature_names_out()

    # calculate initial scores and store them in a dictionary
    scores_dict = {name: score for name, score in zip(feature_names, transformed_data.toarray()[0])}

    # adjust scores based on term length and update the dictionary
    for name in feature_names:
        if name in Mortgage_Terms:
            scores_dict[name] *= (1 + 0.01*len(name) + 1.5)
        else:
            scores_dict[name] *= (1 + 0.01*len(name))

    # create a new array of adjusted scores in the same order as the feature names
    adjusted_scores = np.zeros(len(feature_names))
    for INDEX, term in enumerate(feature_names):
        adjusted_scores[INDEX] = scores_dict[term]

    # return a tuple of the feature names and adjusted scores
    return feature_names, adjusted_scores

def classifiyComplaintTFIDF(user_input):
    # Get an array of words with corresponding scores
    data = tfidf_custom_scoring(user_input)

    # Create a dictionary with feature names as keys and scores as values
    scores_dict = {name: score for name, score in zip(data[0], data[1])}

    # Create a new array of adjusted scores in the same order as the feature names
    adjusted_scores = np.zeros(len(feature_names))
    for i, term in enumerate(feature_names):
        adjusted_scores[i] = scores_dict[term]

    # Set the top 3 words to a list
    Top3Words = []

    def get_top3_words():
        # Get the index of the highest score in the scores array
        index_max = adjusted_scores.argmax()

        while index_max != 0 and len(Top3Words) <= 2:
            # Check if there are any vowels in the top word, if not, select a new word
            while True:
                vowels = {"a", "e", "i", "o", "u", "A", "E", "I", "O", "U"}
                # Get the corresponding word
                top_word = feature_names[index_max]
                if any(char in vowels for char in top_word) and top_word in english_vocab:
                    Top3Words.append(top_word)
                break

            # Get the corresponding word
            top_word = str(data[0][index_max])

            # If the word is in the English vocabulary, add it to the top 3 list
            if top_word in english_vocab:
                Top3Words.append(top_word)

            # Set the current score of this word to 0 to select the second most popular word
            adjusted_scores[index_max] = 0
            index_max = adjusted_scores.argmax()

    get_top3_words()

    # Check all past result classifications
    def check_corresponding_word(relevant_word):
        return data[(data["top_word"].str[0] == relevant_word[0]) & data["top_word"].str[1].isin(relevant_word)]

    filtered_df = check_corresponding_word(Top3Words)

    # Count the occurrences of each issue and get the most common one, as long as there are words left
    value_counts = filtered_df["Issue"].value_counts()
    
    # Concatenate the 'IssueCountNormalized' and 'value_counts' dataframes along the columns axis
    NormalizedTable = pd.concat([IssueCountNormalized, value_counts], axis=1, keys=('perc', 'valuecount'))

    # Calculate the 'Endscores' by dividing the 'valuecount' column by the 'perc' column in 'NormalizedTable'
    Endscores = NormalizedTable.valuecount / NormalizedTable.perc

    # Assign the calculated 'Endscores' as a new column in 'NormalizedTable'
    NormalizedTable["Endscores"] = Endscores

    # Create a new column in 'NormalizedTable' containing the keys from 'IssueCountNormalized' converted to a list
    NormalizedTable["IssueName"] = IssueCountNormalized.keys().tolist()

    # Locate the row in 'NormalizedTable' with the maximum value in the 'Endscores' column
    Toprow = NormalizedTable.loc[NormalizedTable['Endscores'].idxmax()]

    # Return the value in the 'IssueName' column of the row with the highest 'Endscores' value
    return Toprow.IssueName

print("Your question will be in the following category: "+classifiyComplaintTFIDF(input("What question do you want to categorize? ")))


# Articial Intelligence / machine learning methodes

# ChatGPT-2 (Random forest)

In [ ]:
# Read cleaned dataset created from complaintCleaner.py
data = pd.read_csv('StaterData.csv')

# Define your features and target
X = data['Clean consumer complaint']  # text_column is the name of the column in your dataset that contains the text data
y = data['Issue']  # target_column is the name of the column in your dataset that contains the target variable

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Define params from GridSearch which performed the best
param_grid = {
    'n_estimators': 200
}

# Vectorize your text data using TF-IDF vectorizer
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(random_state=2)

# Set the params for the model
rf_model.set_params(**param_grid)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict the target variable for the testing data
y_pred = rf_model.predict(X_test)

def text_to_prediction_random_forest(text):
    custom_text_bow = vectorizer.transform([text])
    predicted_issue = rf_model.predict(custom_text_bow)

    return predicted_issue

# Define your features and target
X = data['Consumer complaint narrative']  # text_column is the name of the column in your dataset that contains the text data
y = data['Issue']  # target_column is the name of the column in your dataset that contains the target variable

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectorize your text data using a bag-of-words approach
vectorizer = TfidfVectorizer(stop_words=['english'])
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=2)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Get feature importances
importances = rf_model.feature_importances_

# Create a DataFrame with feature names and importances
feature_importances = pd.DataFrame({'feature': vectorizer.get_feature_names(), 'importance': importances})

# Sort the DataFrame by importance
feature_importances = feature_importances.sort_values('importance', ascending=False)

top_features = feature_importances.head(10)['feature'].values.tolist()

# Define your features and target
X = data['Consumer complaint narrative']  # text_column is the name of the column in your dataset that contains the text data
y = data['Issue']  # target_column is the name of the column in your dataset that contains the target variable

# Split your data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2)

# Vectorize your text data using a bag-of-words approach
vectorizer = TfidfVectorizer(stop_words='english', vocabulary=top_features)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

# Create a Random Forest model
rf_model = RandomForestClassifier(n_estimators=200, random_state=2)

# Train the model on the training data
rf_model.fit(X_train, y_train)

# Predict the target variable for the testing data
y_pred = rf_model.predict(X_test)

# Evaluate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
f1score = f1_score(y_test, y_pred, average='macro')

# # Define the parameter grid to search over
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [10, 20, 30, None],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
# }
#
# # Create a Random Forest model
# rf_model = RandomForestClassifier(random_state=2)
#
# # Create a GridSearchCV object
# grid_search = GridSearchCV(
#     estimator=rf_model,
#     param_grid=param_grid,
#     cv=5,  # number of folds for cross-validation
#     scoring='accuracy',
#     # n_jobs=-1,  # use all available CPU cores
# )
#
# # Fit the grid search to the training data
# grid_search.fit(X_train, y_train)
# 
# # Print the best hyperparameters and the corresponding F1 score
# print('Best hyperparameters:', grid_search.best_params_)
# print('Best accuracy score:', grid_search.best_score_)

# PaLM 2 (SVM)

In [ ]:
# Print kernal stats
print(f"Python Platform: {platform.platform()}")
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

# Select the right model
palm.configure(api_key='<>')
models = [m for m in palm.list_models() if 'embedText' in m.supported_generation_methods]
model = models[0]

# Import the data, and spit the data
data = pd.read_csv("StaterData.csv")
df_train, df_test = train_test_split(data, test_size=0.3, random_state=2)

# Python progress bar 
tqdm.pandas()

def make_embed_text_fn(model):
  @retry.Retry(timeout=300.0)
  def embed_fn(text: str) -> list[float]:
    # Using the palm model generate the embeddings for 
    return palm.generate_embeddings(model=model, text=text)['embedding']
  return embed_fn

# Creates the embedding of the send dataframe
def create_embeddings(model, data):
  # Adds the column embeddings with the corrosponding embedding
  data['Embeddings'] = data['Clean consumer complaint'].progress_apply(make_embed_text_fn(model))
  return data

# Creates the embedding for the train and the testdata
df_train = create_embeddings(model, df_train)
df_test = create_embeddings(model, df_test)

# Calling the API from Palm and convert the text to vectors
def build_classification_model(input_size: int, num_classes: int) -> keras.Model:
  inputs = x = keras.Input(input_size)
  x = layers.Dense(input_size, activation='relu')(x)
  x = layers.Dense(num_classes, activation='sigmoid')(x)
  return keras.Model(inputs=[inputs], outputs=x)

# Derive the embedding size from the first training element.
embedding_size = len(df_train['Embeddings'].iloc[0])

# Give your model a different name, as you have already used the variable name 'model'
classifier = build_classification_model(embedding_size, len(df_train['Clean consumer complaint'].unique()))
classifier.summary()

classifier.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                   optimizer = keras.optimizers.Adam(learning_rate=0.001),
                   metrics=['accuracy'])

# Tensorflow model training
NUM_EPOCHS = 20
BATCH_SIZE = 8

# Split the x and y components of the train and validation subsets.
y_train = df_train['Issue_Int']
x_train = np.stack(df_train['Embeddings'])
y_val = df_test['Issue_Int']
x_val = np.stack(df_test['Embeddings'])

# Train the model for the desired number of epochs.
callback = keras.callbacks.EarlyStopping(monitor='accuracy', patience=3)

history = classifier.fit(x=x_train,
                         y=y_train,
                         validation_data=(x_val, y_val),
                         callbacks=[callback],
                         batch_size=BATCH_SIZE,
                         epochs=NUM_EPOCHS,) 

classifier.evaluate(x=x_val, y=y_val, return_dict=True)
